# Convolutional Neural Net

In this workbook we'll apply a CNN to the image data that was generated in the previous workbook.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import pickle
import time
import os
import json
import seaborn as sns
import tensorflow as tf
import random


from pandas import json_normalize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score,classification_report
from sklearn.preprocessing import normalize,StandardScaler,RobustScaler,MinMaxScaler
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.compose import ColumnTransformer


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, Conv2D, MaxPooling2D,LeakyReLU
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import mnist, cifar10
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model




import librosa
import librosa.display

from scipy.stats import skew

### Importing Variables

In [2]:
with open('pickles/Y_low.pkl', 'rb') as f:
    Y_low = pickle.load(f)
    
with open('pickles/X_low.pkl', 'rb') as f:
    X_low = pickle.load(f)

### Random Sampling

In [3]:
len(X_low),len(Y_low)

(68566, 68566)

### Train-test Split

In [4]:
X = np.array(X_low).astype('float32') / 255.0
y = np.array(Y_low)

In [5]:
X[0][0].shape[0]

224

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
X_train.shape

(54852, 224, 224, 3)

In [8]:
X_train = X_train.reshape(X_train.shape[0], 224, 224, 3)
X_val = X_val.reshape(X_val.shape[0], 224, 224, 3)

### Autoencoder

In [9]:
input_img = Input(shape=(224, 224, 3))
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

Metal device set to: Apple M1 Max


2023-04-21 17:45:55.659057: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-21 17:45:55.659953: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
# Train the autoencoder on the spectrogram data
autoencoder.fit(X_train, X_train,
                epochs=10,
                batch_size=128,
                shuffle=True,
                validation_data=(X_val, X_val))

# Extract features using the learned encoder
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('max_pooling2d_2').output)
X_train_encoded = encoder.predict(X_train)
X_val_encoded = encoder.predict(X_val)

In [13]:
tf.config.list_physical_devices('mps')

[]

In [14]:
model = Sequential([
    Conv2D(64, (3, 3), strides=(2, 2), activation=LeakyReLU(alpha=0.1), input_shape=(X_train.shape[1:])),
    MaxPooling2D(2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(.2),
    Dense(2, activation='sigmoid')
])

2023-04-21 17:43:55.500796: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-21 17:43:55.502270: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Max


In [15]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [16]:
early_stopping = EarlyStopping(monitor='val_loss', patience=6)

In [ ]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    batch_size = 32,
    epochs = 100,
    verbose= 1,
    callbacks=[early_stopping]
)

In [ ]:
yellow = '#f8c84c'
grey = '#4F4F4F'
background = '#e8e4e4'

# Create the figure with the grey background
fig = plt.figure(figsize=(8, 6), facecolor=background)

# Set the background color of the plot area to grey
ax = fig.add_subplot(111)
ax.set_facecolor(background)

# Plot the data with the yellow-grey color scheme
plt.plot(history.epoch, history.history['loss'], color=yellow, label='Training Loss')
plt.plot(history.epoch, history.history['val_loss'], color=grey, label='Validation Loss')
plt.yscale('linear')
plt.title('Training & Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Show the plot
plt.show()

In [ ]:
with open('pickles/model.pkl', 'wb') as f:
    pickle.dump(model, f)